In [76]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [39]:
df = pd.read_csv('availability_and_weather.csv')

/tmp/ipykernel_648965/719539868.py:1: DtypeWarning: Columns (14,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('availability_and_weather.csv')


In [42]:
df.columns

Index(['STATION ID', 'TIME', 'LAST UPDATED', 'NAME', 'BIKE_STANDS',
       'AVAILABLE_BIKE_STANDS', 'AVAILABLE_BIKES', 'STATUS', 'ADDRESS',
       'LATITUDE', 'LONGITUDE', 'DAY', 'HOUR', 'AVAILABILITY', 'date', 'ind',
       'rain', 'ind.1', 'temp', 'ind.2', 'wetb', 'dewpt', 'vappr', 'rhum',
       'msl', 'ind.3', 'wdsp', 'ind.4', 'wddir', 'ww', 'w', 'sun', 'vis',
       'clht', 'clamt'],
      dtype='object')

In [88]:
stations = df['STATION ID'].unique()

In [43]:
df = df.dropna()

In [89]:
for station_num in stations:
    df_station = df[df['STATION ID'] == station_num]
    y = df_station['AVAILABILITY']
    days_hours = df_station[['DAY', 'HOUR', 'rain', 'temp', 'rhum']]
    X = pd.get_dummies(days_hours)

    scaler = StandardScaler()

    scaled_X = scaler.fit_transform(X)

    # poly = PolynomialFeatures(degree=4)
    # X_poly = poly.fit_transform(X)
    # X_poly

    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42)
    

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Station
    print(station_num)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

    # Calculate R-squared (R2)
    r2 = r2_score(y_test, y_pred)
    print("R-squared (R2):", r2)

    # Save the model to pickle
    filename = f'models/{station_num}.pkl'
    with open(filename,'wb') as handle:
        pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)

1
Mean Squared Error: 0.07551067634323158
R-squared (R2): 0.19627992347612921
2
Mean Squared Error: 0.06449985604168541
R-squared (R2): 0.13665203515804547
3
Mean Squared Error: 0.07883329257433648
R-squared (R2): 0.09448054407480144
4
Mean Squared Error: 0.0742257417360556
R-squared (R2): 0.07963402401089503
5
Mean Squared Error: 0.10169640035658037
R-squared (R2): 0.14526256852019337
6
Mean Squared Error: 0.06642656241135288
R-squared (R2): 0.35272042579814233
7
Mean Squared Error: 0.055657853572905175
R-squared (R2): 0.048867563885947374
8
Mean Squared Error: 0.06618908338431645
R-squared (R2): 0.06408003195839274
9
Mean Squared Error: 0.07771482979928651
R-squared (R2): 0.25849901254791785
10
Mean Squared Error: 0.06645106898523033
R-squared (R2): 0.193203570651737
11
Mean Squared Error: 0.051157360874599066
R-squared (R2): 0.10533080296437136
12
Mean Squared Error: 0.048779822809958524
R-squared (R2): 0.10687150938620715
13
Mean Squared Error: 0.041281519855425414
R-squared (R2): 

In [74]:
# import model for depart station
with open(f'./models/{station_num}.pkl', 'rb') as file:
    model = pickle.load(file)

In [84]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(np.asarray([12,0,0,1,0,0,0,0,23,12,92]).reshape(1,-1))

model.predict(scaled_X)

array([0.44323971])

In [60]:
df_station['rhum'].value_counts()

rhum
91.0    1098
88.0    1058
90.0    1037
89.0     984
92.0     978
        ... 
33.0       4
35.0       2
30.0       2
37.0       2
34.0       2
Name: count, Length: 68, dtype: int64

In [4]:
import requests
from datetime import datetime
import json
from constants import * 

In [18]:

print(f"Fetching weather data from API at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
headers = {"accept": "application/json"}
url = f"http://api.weatherapi.com/v1/forecast.json?key={WEATHER_API_KEY}=Dublin&aqi=no&alerts=no" # 7 day forecast including realtime
response = requests.get(url, headers=headers)
response_dictionary = response.json()
global weather_data_cache # Use global variable to store the weather data
weather_data_cache = response_dictionary
if response.status_code == 200:
    print("Successful fetch from weather API.")
    current_data = weather_data_cache['current']
    weather_data = {
        'last_updated': current_data['last_updated_epoch'],
        'rain': current_data['precip_mm'],
        'temp': current_data['temp_c'],
        'hum': current_data['humidity']
    }
    print(json.dumps(weather_data, indent=4))  # Adjusted to return JSON for API endpoint
    print(response_dictionary)
else:
    error_message = f"Failed to fetch from weather API. Status Code: {response.status_code}"
    print(error_message)

Fetching weather data from API at 2024-04-09 08:21:26
Successful fetch from weather API.
{
    "last_updated": 1712646900,
    "rain": 0.01,
    "temp": 5.0,
    "hum": 87
}
{'location': {'name': 'Dublin', 'region': 'Dublin', 'country': 'Ireland', 'lat': 53.33, 'lon': -6.25, 'tz_id': 'Europe/Dublin', 'localtime_epoch': 1712647285, 'localtime': '2024-04-09 8:21'}, 'current': {'last_updated_epoch': 1712646900, 'last_updated': '2024-04-09 08:15', 'temp_c': 5.0, 'temp_f': 41.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 21.7, 'wind_kph': 34.9, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1005.0, 'pressure_in': 29.68, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 87, 'cloud': 75, 'feelslike_c': -0.1, 'feelslike_f': 31.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 2.0, 'gust_mph': 24.5, 'gust_kph': 39.4}, 'forecast': {'forecastday': [{'date': '2024-04-09', 'date_epoch': 1712620800, 'day': {'m